In [16]:
import pandas as pd
import numpy as np

In [17]:
!git clone https://github.com/9berny9/bikes

fatal: destination path 'bikes' already exists and is not an empty directory.


# BIKES IN EDINBURGH
1. [Zadání](#1)
2. [Úprava dat](#2)
3. [Analýza](#3)



## Zadání <a name="1"></a>



V Edinburghu, stejně jako v dalších městech, funguje systém "bike sharing" - ve městě jsou stanice s koly, člověk si může nějaké půjčit a potom ho vrátit v nějaké další stanici. Problém je, že v některých stanicích se kola pravidelně hromadí a jinde naopak chybí. Provozovatel kol, firma Just Eat Cycles, zadala projekt, jehož cílem je systém zefektivnit.

Coby datový analytik jste součástí týmu, který na projektu pracuje. Vaším úkolem je zpracovat relevantní data a zjistit z nich informace užitečné pro zbytek týmu. Máte k dispozici data o všech výpůjčkách (na ENGETO databázi v tabulce edinburgh_bikes). Proveďte standardní deskriptivní statistiku dat. Také zjistěte minimálně následující informace:

identifikujte aktivní a neaktivní stanice
identifikujte nejfrekventovanější stanice
identifikujte stanice, na kterých se kola hromadí a stanice, kde potenciálně chybí
spočítejte vzdálenosti mezi jednotlivými stanicemi
jak dlouho trvá jedna výpůjčka? Najděte odlehlé hodnoty, zobrazte histogram
Analýza poptávky:

zobrazte vývoj poptávky po půjčování kol v čase
identifikujte příčiny výkyvů poptávky
zjistěte vliv počasí na poptávku po kolech (údaje o počasí v Edinburghu jsou v tabulce edinburgh_weather)
půjčují si lidé kola více o víkendu než během pracovního týdne?
Výstupem analýzy bude ideálně jupyter notebook nebo colab notebook uložený na Vašem githubu. Výstupy Vaší analýzy musí být zřejmé i bez spouštění kódu. V textových buňkách popište, jak jste postupovali a napište své závěry. Pokud v zadání chybí nějaká informace, doplňte ji podle vlastního uvážení, své rozhodnutí uveďte a vysvětlete v notebooku.

## Úprava dat <a name="2"></a>

* Veškerá práce s databází a úprava datových formátů se nachází v repozitáři zde: https://github.com/9berny9/bikes/blob/90970fb641920d89a08362277943fd1b273625eb/data_process.ipynb

* Výstupem úpravy dat jsou pickle soubory uložené v repozitáři **bikes/Database/modified_data**, na kterých probíhá celá analýza.

## Analýza<a name="3"></a>

### Stanice<a name="3.1"></a>

In [165]:
# načtení pickle do dataframu
url_stations = "/content/bikes/Database/modified_data/stations.pkl"
df_s = pd.read_pickle(url_stations)

In [91]:
# deskriptivní statistika
df_s[["start_frequency", "end_frequency"]].describe()

,start_frequency,end_frequency
count,198.000000,199.000000
mean,2213.429293,2202.306533
std,2884.099842,2898.879223
min,1.000000,2.000000
25%,257.000000,236.000000
50%,1064.500000,1231.000000
75%,3154.500000,2892.000000
max,17390.000000,16656.000000


V Edinburghu se nachází **200** stanic pro půjčení kol a data jsou sbírány od **2018-09-15** do **2021-06-30**, což je **1019** dnů.

*   Průměrný počet půjčených kol na stanici je **2213 výpujček kol**, což je **2,17** výpujček za den.
*   Medián počtu půjčených kol na stanici je **1064 výpujček kol**, což je **1,04** výpujček za den.
*   Průměrný počet vrácených kola na stanici je **2202 vrácení kol**, což je **2,16** výpujček za den.
*   Medián počtu vrácených kol na stanici je **1231 vrácení kol**, což je **1,21** výpujček za den.
*   Minimální hodnota počtu výpujček ve stanicích je **1** a počtu vrácení je **2** za celé období.   Tyto stanice můžeme označit jako neaktivní a budou řešeny v pozdější analýze).
*   Maximální hodnota počtu výpujček je **17390**, což je **17,07** výpujček za den a maximální hodnota počtu vratek je **16656**, což je **16,35** vratek za den.


#### Mapa stanic<a name="3.1.1"></a>

In [61]:
import plotly.express as px

In [63]:
# vykreslení mapy stanic
fig = px.scatter_mapbox(df_s,lat="lat", lon="lon", hover_name = "name", hover_data = ["description","start_frequency","end_frequency"] ,zoom=10, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#### Histogram využívání stanic<a name="3.1.1"></a>

In [88]:
# vytvoření histogramu podle frekvence počtu vypůjčení
px.histogram(df_s, x="start_frequency")

In [166]:
# nahradit řádky NaN za 0
df_s["start_frequency"] = df_s.start_frequency.fillna(0)

In [185]:
# interquartile range
Q1 = df_s["start_frequency"].quantile(0.25)
Q3 = df_s["start_frequency"].quantile(0.75)
IQR = Q3 - Q1

In [187]:
Q3

3119.5

In [179]:
# odlehlé hodnoty
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1,5 * IQR

In [183]:
lower_bound

-4077.375

In [180]:
df_s[(df_s["start_frequency"] < lower_bound) | (df_s["start_frequency"] > upper_bound)]

ValueError: ignored

In [ ]:
first_quartil = 221
third_quartil = 4053
IQR = third_quartil - first_quartil                                 # Interquartilový rozsah
outlier = round(1.5 * IQR + third_quartil)
strong_outlier = round(3 * IQR + third_quartil)
print("Hranice slabé odlehlé hodnoty je: ", outlier, "x")
print("Hranice silné odlehlé hodnoty je: ", strong_outlier, "x")
print("")

df_outlier = df_start_stations_frequency.query("frequency_start_station > 9801 and frequency_start_station <= 15549").count()
print("POČET SLABÝCH ODLEHLÝCH HODNOT: ", df_outlier["frequency_start_station"])
df_strong_outlier = df_start_stations_frequency.query("frequency_start_station > 15549").count()
print("POČET SILNÝCH ODLEHLÝCH HODNOT: ", df_strong_outlier["frequency_start_station"])

#### Vzdálenosti stanic<a name="3.1.2"></a>

In [48]:
from sklearn.metrics import DistanceMetric

In [75]:
# kopie dataframu pro výpočet vzdáleností seřazená podle id
df_s2 = df_s.copy().sort_values(by="id")

postup výpočtu vzdáleností:   
https://medium.com/@danalindquist/finding-the-distance-between-two-lists-of-geographic-coordinates-9ace7e43bb2f

In [76]:
# převedení na radiány
df_s2['lat'] = np.radians(df_s2['lat'])
df_s2['lon'] = np.radians(df_s2['lon'])

In [80]:
# vytvoření matice pomocí knihovny
dist = DistanceMetric.get_metric('haversine')

In [81]:
# radius země v km
earth_radius = 6373
# matice s výpočtem vzdálenosti mezi stanicemi v km
distance = pd.DataFrame(dist.pairwise(
    (df_s2[['lat','lon']]) * earth_radius).round(2),
      columns=df_s2.id, index=df_s2.id)

In [83]:
distance.head()

id,171,183,189,225,241,242,246,247,248,249,...,1868,1869,1870,1871,1874,1877,2259,2263,2265,2268
id,,,,,,,,,,,,,,,,,,,,,
171,0.00,0.96,0.80,1.03,1.84,1.84,0.64,1.92,0.33,1.15,...,1.07,0.89,0.23,1.09,1.30,0.70,2.47,0.58,1.05,1.51
183,0.96,0.00,0.19,0.81,1.13,1.13,1.59,1.75,0.66,1.78,...,1.58,1.60,1.18,0.57,1.36,1.59,1.51,1.37,1.95,0.80
189,0.80,0.19,0.00,0.70,1.18,1.18,1.44,1.88,0.48,1.74,...,1.56,1.53,1.02,0.69,1.43,1.46,1.67,1.18,1.76,0.83
225,1.03,0.81,0.70,0.00,0.83,0.83,1.51,2.54,0.79,2.18,...,2.09,1.92,1.22,1.37,2.11,1.69,1.65,0.97,1.47,0.58
241,1.84,1.13,1.18,0.83,0.00,0.00,2.33,2.10,1.54,2.91,...,2.69,2.70,2.04,1.54,2.28,2.52,0.85,1.76,2.08,0.35
